In [103]:
import requests 
import json
import pandas as pd
from pprint import pprint
import sys

In [104]:
def merge_url(var):
    url = 'http://rest.ensembl.org/vep/human/hgvs/'
    new_url = url + var
    return new_url

In [142]:
def req_new_url(new_url):
    object1 = requests.get(new_url,headers={ "Content-Type" : "application/json"})
    if not object1.ok:
        #object1.raise_for_status() 
        return -2
    try: 
        object2 = object1.json()
    except json.JSONDecodeError:
        return -1
    return object2

In [106]:
def txt_to_list(filepath):
    with open(filepath) as f:
        variants = []
        for line in f:
            stripped_line = line.strip()
            variants.append(stripped_line)
    return variants 
        

In [145]:
def get_tsv(file_name):
    list_of_variants = txt_to_list(file_name)
    error_api = [] #To store variants which return error #Print the error API variants at the end
    rows = []
    #df = pd.DataFrame() #New DataFrame
    for var in list_of_variants:  #Loop running through individual variants
        url = merge_url(var) #Creates API url for each variants
        obj = req_new_url(url)  #Object which stores JSON object for individual Campaign IDs
        if type(obj) == int :
            pass
            error_api.append(var)           
        else :
            for key in obj:
                temp = []
                temp_conseq = []
                temp.append(var)
                temp.append(key['assembly_name'])
                temp.append(key['seq_region_name'])
                temp.append(key['start'])
                temp.append(key['end'])
                temp.append(key['most_severe_consequence'])
                temp.append(key['strand'])
                for key_2 in key['transcript_consequences']:                    
                    temp_conseq.append(key_2['gene_symbol'])
                temp_conseq_set = set(temp_conseq)
                temp_conseq = list(temp_conseq_set)
                temp.append(temp_conseq)
                rows.append(temp)
    df = pd.DataFrame(rows,columns=['input_variant','assembly_name','seq_region_name','start','end','most_severe_consequence','strand','transcript_consequences'])
    print(df)        
    
    
    

In [147]:
pprint(get_tsv('variants.txt'))

                 input_variant assembly_name seq_region_name      start  \
0  NC_000001.11:g.215674515G>A        GRCh38               1  215674515   
1   NC_000001.11:g.40819893G>A        GRCh38               1   40819893   
2   NC_000002.12:g.39006443C>T        GRCh38               2   39006443   
3  NC_000006.12:g.152387156G>A        GRCh38               6  152387156   
4  NC_000001.11:g.215674515G>A        GRCh38               1  215674515   

         end most_severe_consequence  strand transcript_consequences  
0  215674515        missense_variant       1                 [USH2A]  
1   40819893        missense_variant       1                 [KCNQ4]  
2   39006443      synonymous_variant       1                  [SOS1]  
3  152387156      synonymous_variant       1                 [SYNE1]  
4  215674515        missense_variant       1                 [USH2A]  
None
